# Cycling (Crypto)

> It is well known that any RSA encryption can be undone by just encrypting the ciphertext over and over again.  
> If the RSA modulus has been chosen badly then the number of encryptions necessary to undo an encryption is small.  
> However, if the modulus is well chosen then a cycle attack can take much longer. This property can be used for a timed release of a message.  
> We have confirmed that it takes a whopping 2^1025-3 encryptions to decrypt the flag.  
> Pack out your quantum computer and perform 2^1025-3 encryptions to solve this challenge. Good luck doing this in 48h.  
> 
> [Attachment](https://storage.googleapis.com/gctf-2022-attachments-project/31936a1acfb1f3206c23c02aee31e79846c4c3fe34c7dacb3f8ee67596ef42fd48b8d1462b88455e8d22f93a49d91d2939c30559d4e36803b2f5efd1b9a2fd40)
> 
> 50 Solves, 201 points

We get a python script which describes a RSA cryptosystem with $n$, $e$ and $\text{ct}$, which also describes the cycling operation needing $2^{1025} - 3$ encryptions to get the flag.

```python
e = 65537
n = 0x99efa91773[...2048 bits...]8b552b1
ct = 0x339be51512[...2048 bits...]6a945076524877680
# Decryption via cycling:
pt = ct
for _ in range(2**1025 - 3):
  pt = pow(pt, e, n)
# Assert decryption worked:
assert ct == pow(pt, e, n)

# Print flag:
print(pt.to_bytes((pt.bit_length() + 7)//8, 'big').decode())
```

The first step is to notice that repeat encryptions can also be represented as raising $ct$ to some power of $e$:

$$
\begin{align}
((((\text{ct} ^ e) ^ e) ^ e) ^ \cdots) &= \text{ct} ^ {(e ^ {2^{1025} - 3})} \\
&= \text{pln} \mod n
\end{align}
$$

In [1]:
e = 65537
n = 0x99efa9177387907eb3f74dc09a4d7a93abf6ceb7ee102c689ecd0998975cede29f3ca951feb5adfb9282879cc666e22dcafc07d7f89d762b9ad5532042c79060cdb022703d790421a7f6a76a50cceb635ad1b5d78510adf8c6ff9645a1b179e965358e10fe3dd5f82744773360270b6fa62d972d196a810e152f1285e0b8b26f5d54991d0539a13e655d752bd71963f822affc7a03e946cea2c4ef65bf94706f20b79d672e64e8faac45172c4130bfeca9bef71ed8c0c9e2aa0a1d6d47239960f90ef25b337255bac9c452cb019a44115b0437726a9adef10a028f1e1263c97c14a1d7cd58a8994832e764ffbfcc05ec8ed3269bb0569278eea0550548b552b1
ct = 0x339be515121dab503106cd190897382149e032a76a1ca0eec74f2c8c74560b00dffc0ad65ee4df4f47b2c9810d93e8579517692268c821c6724946438a9744a2a95510d529f0e0195a2660abd057d3f6a59df3a1c9a116f76d53900e2a715dfe5525228e832c02fd07b8dac0d488cca269e0dbb74047cf7a5e64a06a443f7d580ee28c5d41d5ede3604825eba31985e96575df2bcc2fefd0c77f2033c04008be9746a0935338434c16d5a68d1338eabdcf0170ac19a27ec832bf0a353934570abd48b1fe31bc9a4bb99428d1fbab726b284aec27522efb9527ddce1106ba6a480c65f9332c5b2a3c727a2cca6d6951b09c7c28ed0474fdc6a945076524877680

F = Zmod(n)

repeat_encryptions = F(ct)
for i in range(5):
    repeat_encryptions = repeat_encryptions ^ e

single_exponent = F(ct) ^ (e ^ 5)

assert repeat_encryptions == single_exponent

We can also replace $\text{ct}$ with $\text{pln}^e$ to remove one unknown.

$$
\begin{align}
({\text{pln} ^ e}) ^ {(e ^ {2^{1025} - 3})} &= \text{pln} ^ {e \cdot (e ^ {2^{1025} - 3})} \\
&=  \text{pln} ^ {(e ^ {2^{1025} - 2})} \\
&= \text{pln} \mod n
\end{align}
$$

With a common base of $\text{pln}$, we can work with the exponents modulo $\lambda (n)$, where $\lambda$ is the Carmichael lambda function.

$$
\begin{align}
e ^ {2 ^ {1025} - 2} &= 1
&= e ^ 0  \mod \lambda (n)
\end{align}
$$

And then assuming $e$ is coprime to $\lambda (n)$, we can change once more to work with the exponents modulo $\lambda ( \lambda (n))$*.

$$
\begin{align}
2 ^ {1025} - 2 &= 0  \mod \lambda ( \lambda (n)) \\
2 ^ {1025} - 2 &= k \cdot \lambda ( \lambda (n))
\end{align}
$$

From this manipulation, we have found that $2 ^ {1025} - 2$ is some multiple of $\lambda ( \lambda (n))$. We can verify this by analyisng the toy example given earlier in the attachment.

\* Not exactly, see [footnote](./cycling_footnote.png)

In [2]:
from sage.crypto.util import carmichael_lambda

"""
>>> e = 65537
>>> n = 0x112b00148621
>>> pt = 0xdeadbeef
>>> # Encryption
>>> ct = pow(pt, e, n)
>>> # Decryption via cycling:
>>> pt = ct
>>> for _ in range(209):
>>>   pt = pow(pt, e, n)
>>> # Assert decryption worked:
>>> assert ct == pow(pt, e, n)
"""

e = 65537
example_n = 0x112b00148621
print(example_n, "=", example_n.factor())
lm = carmichael_lambda(example_n)
print(lm, "=", lm.factor())
lmlm = carmichael_lambda(lm)
print(lmlm, "=", lm.factor())

# Script suggests 209 + 1 is a multiple of lambda(lambda(n)). In fact, 210 is exactly it.

18876382610977 = 3021943 * 6246439
9225988926 = 2 * 3 * 7 * 11 * 31 * 43 * 71 * 211
210 = 2 * 3 * 7 * 11 * 31 * 43 * 71 * 211


To move on, we need to compute some $x$ where $\lambda (x) = 2 ^ {1025} - 2$, i.e. reverse the carmichael lambda function. As this is very dependent on the factorisation of $\lambda (x)$, a quick search on [factordb](http://factordb.com/index.php?query=359538626972463181545861038157804946723595395788461314546860162315465351611001926265416954644815072042240227759742786715317579537628833244985694861278948248755535786849730970552604439202492188238906165904170011537676301364684925762947826221081654474326701021369172596479894491876959432609670712659248448274430) was done and it gave an interesting result.

![factordb](factordb.png)

Not only is $2 ^ {1025} - 2$ fully factorised (lets denote these prime factors as $q_i$), but all primes in its factorization do not have power $2$ or greater. Looking at the definition of the carmichael function for compound numbers (taken off [wikipedia](https://en.wikipedia.org/wiki/Carmichael_function#Computing_%CE%BB(n)_with_Carmichael's_theorem)):

$$
\begin{align}
\lambda(n) &= \operatorname{lcm}\Bigl(\lambda\left(p_1^{r_1}\right),\lambda\left(p_2^{r_2}\right),\ldots,\lambda\left(p_k^{r_k}\right)\Bigr) \\
\lambda(p^r) &=
\begin{cases}
\tfrac12\varphi\left(p^r\right)&\text{if }p=2\land r\geq 3 \;(\text{i.e. }p^r = 8,16,32,64,128,256,\dots)\\
\varphi\left(p^r\right) &\text{otherwise}\;(\text{i.e. }p^r = 2,4,3^r,5^r,7^r,11^r,13^r,17^r,19^r,23^r,29^r,31^r,\dots)
\end{cases} \\
\varphi\left(p^r\right) &= p^{r-1} \cdot (p - 1)
\end{align}
$$

This suggests that all prime factors $p_i$ of $x$ must not have power $3$ or more (or some $q_i^2$ would have appeared in the factorisation of $\lambda(x)$). Further analysis of the prime factors of $\lambda(x)$ also shows that products of small prime factors of $\lambda(x)$ are insufficient for the $p_i \cdot (p_i - 1)$ case of the euler $\varphi$ to occur, thus the prime factors of $x$ must also not have power $2$. All $p_i$ must therefore be in the form of $p_i = 1 + 2 \cdot q_a \cdot q_b \cdots$. Since there are 16 $q_i$ not including $2$, we can brute force search over the $2^{16}$ subsets of $q_i$ to find all possible primes $p_i$. Since the chance of finding primes tends towards $\frac{1}{\log(n)}$, we can expect to find around $5000$ primes (but in reality much less).

In [3]:
from gmpy2 import mpz
from itertools import product as iter_product
from tqdm.auto import tqdm

qis = [
    2, 3, 5, 17, 257, 641, 65537, 274177,
    2424833, 6700417, 67280421310721,
    1238926361552897, 59649589127497217,
    5704689200685129054721,
    7455602825647884208337395736200454918783366342657,
    (2 ^ 256 + 1) // 1238926361552897,
    (2 ^ 512 + 1) // 18078591766524236008555392315198157702078226558764001281
]
assert product(qis) == 2^1025 - 2
print("Number of q_i =", len(qis))

qi_choices = []
for qi in qis:
    if qi == 2:
        # qi_choice = [2]
        qi_choice = [mpz(int(2))]
    else:
        # qi_choice = [1, qi]
        qi_choice = [mpz(int(1)), mpz(int(qi))]
    qi_choices.append(qi_choice)
    
total = product(len(i) for i in qi_choices)
print("Brute forcing count =", total)
print("Expected primes =", numerical_approx(total / log(total)))

possible_pis = [mpz(2), mpz(2), mpz(2)] # just to be safe
for qi_subset in tqdm(iter_product(*qi_choices), total=total):
    possible_pi = product(qi_subset) + 1
    if possible_pi.is_prime():
        possible_pis.append(possible_pi)
len(possible_pis)
print(possible_pis[-5:])

Number of q_i = 17
Brute forcing count = 65536
Expected primes = 5909.27888748119


  0%|          | 0/65536 [00:00<?, ?it/s]

[mpz(513710392204877389479761524744363286606118849589478726911), mpz(3830020651687366694552285361402374194495621331872404170319661309366026837199635344945866679546591186799871), mpz(1212950520775228784418380499921882761764237233342238168508449918139581154095034822761871277222057121635383704669501434740296643199446098044944544292256602314426015983640459973450744608511), mpz(18190965659117862716608733189329171361877919823696689981673601407887800424069245071976565548724930718685265405727088121640041574911), mpz(1812656540844077372157219246333249048567961587608217731442755071)]


All prime factors of $x$ i.e. $\lambda (n)$ will be within the list of found `possible_pi`. The product of all found primes will therefore be some multiple of $\lambda (n)$, which also means that we have sufficient information to generate the RSA private key. We can then decrypt the ciphertext.

In [4]:
many_multiple_of_lambda_n = product(possible_pis)
print("Lambda bit length =", many_multiple_of_lambda_n.bit_length())

d = pow(e, -1, int(many_multiple_of_lambda_n))
ct = 0x339be515121dab503106cd190897382149e032a76a1ca0eec74f2c8c74560b00dffc0ad65ee4df4f47b2c9810d93e8579517692268c821c6724946438a9744a2a95510d529f0e0195a2660abd057d3f6a59df3a1c9a116f76d53900e2a715dfe5525228e832c02fd07b8dac0d488cca269e0dbb74047cf7a5e64a06a443f7d580ee28c5d41d5ede3604825eba31985e96575df2bcc2fefd0c77f2033c04008be9746a0935338434c16d5a68d1338eabdcf0170ac19a27ec832bf0a353934570abd48b1fe31bc9a4bb99428d1fbab726b284aec27522efb9527ddce1106ba6a480c65f9332c5b2a3c727a2cca6d6951b09c7c28ed0474fdc6a945076524877680
pln = pow(ct, d, n)
print(pln)

print(bytes.fromhex(hex(int(pln))[2:]))

Lambda bit length = 225657
6448856217743044601730933741207105905970494861939405949
b'CTF{Recycling_Is_Great}'


**Extra credit**: we can recover p and q from our multiple of $\lambda (n)$ to take a look at its factorisation.

In [5]:
actually_in_lambda = []

good_count = 0
while True:
    cur = F(randint(2, n))
    for i in actually_in_lambda:
        cur = cur ^ i

    if cur == 1:
        if good_count == 10:
            break
        else:
            good_count += 1

    for i in possible_pis:
        cur = cur ^ i
        if cur == 1:
            actually_in_lambda.append(i)
            break

print(actually_in_lambda)
lm = product(actually_in_lambda)
print("lambda =", lm)

[mpz(84647865579983770182156840435960323589220745705078804840882173545543161971711), mpz(378200141805046496283379438166173335669961950663539756272918309398604683306094658372411045924118879925239716104152356497298164557390803047), mpz(8289601665343631680694136683126283913490535883046626291297985618766420841670476534815478471133446480072725379211419560217759461223), mpz(16610502701712857967225573619052807325007025304374821866085832895214500447730251636128678226944940213168394241646062599), mpz(112294839769470698482415380891733048574606244662664090132497671500178069462250958484664109862998738091), mpz(9814268015128689158310993113420627722711154551137827), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2), mpz(2)]
lambda = 99495125319527570444567619875477064158599250193627778435242261737760983473546138838151272368969075861168652786534716529032067825210691446789880538393832019345398222771549210480386235299610305892093164166870107314154228492876673411496857322743

In [6]:
k = lm
p, q = None, None
while True:
    t = k
    g = randint(2, n - 1)

    while t % 2 == 0:
        t = t // 2
        x = ZZ(Zmod(n)(g) ^ t)
        if gcd(x - 1, n) > 1:
            p = gcd(x - 1, n)
            q = n // p

    if p is not None:
        break
p, q

(123308317792004108044227075660012978992959806682575644399876015151433488334825111684207930755121481090869455222578120192795560645012395286103522809837527423661898532411193938620043357379228521383739100318575754223572387736809743299919341601216883337279391593374062125329975995371532041765220472096870101281063,
 157593924010455786071298640263840808220664896044845220551688298727602115308126289164503989738493339695181396796167460601134708107699736784103133872042074350526304685343020287340388227158273950959362140925065432662926545098212293156230985422779937297612188720284623808489935491200722403614227642081332914412647)